Chcu:
- *ofiko název*
- *bio*
- x žánr
- x kapela/jednotlivec? (Ale bacha u interpret_hlavni!)
- *země původu (co když víc?)*
- x pohlaví
- *etnicita?*
- x rok narození 
- *místo narození*
- x rok úmrtí
- *místo úmrtí*
- x začátek kariéry
- x konec kariéry

Možná ještě:
- last.fm id
- x spotify id
- ocenění?

In [217]:
import os
import json
import requests

In [218]:
from wikidata.client import Client
import wikidata.entity

In [219]:
with open(os.path.join("wikidata","wiki_ids.json"), "r") as file:
    qecka = json.load(file)

In [7]:
def wikidates(id):
    client = Client()
    entity = client.get(id, load=True)
    return entity

In [8]:
def vsechny_atributy(q,p):
    atributy = []
    for x in wikidates(q).attributes['claims'][p]:
        atributy.append(x['mainsnak']['datavalue']['value']['id'])
    return(atributy)

In [9]:
def get_wikidata_item_label(wikidata_id, language):
    url = f'https://www.wikidata.org/wiki/Special:EntityData/{wikidata_id}.json'
    response = requests.get(url)
    data = response.json()
    label = data['entities'][wikidata_id]['labels'][language]['value']
    return label

In [79]:
get_wikidata_item_label('Q83270', 'cs')

'hard rock'

In [13]:
get_wikidata_item_label('Q163829', 'cs')

'syntezátor'

## Atributy všech

### Popisy

pseudonym (P742)

In [15]:
wikidates('Q19848').attributes['claims']['P742'][0]['mainsnak']['datavalue']['value']

'Lady Gaga'

subject named as (P1810)
name by which a subject is recorded in a database, mentioned as a contributor of a work, or is referred to in a particular context

In [251]:
wikidates('Q6774073').attributes['claims']['P691'][0]['qualifiers']['P1810'][0]['datavalue']['value']

'Kryštof (hudební skupina)'

#### Žánr já respektuji

In [17]:
vsechny_atributy('Q45188','P136')

['Q11366', 'Q484641', 'Q37073', 'Q95469495', 'Q217191', 'Q3231749']

#### Spotify ID

In [18]:
wikidates('Q19848').attributes['claims']['P1902'][0]['mainsnak']['datavalue']['value']

'1HY2Jd0NmPuamShAr6KMms'

#### Spotify odběratelstvo

In [19]:
wikidates('Q19848').attributes['claims']['P1902'][0]['qualifiers']['P3744'][0]['datavalue']['value']['amount']

'+20502345'

#### Začátek a konec kariéry

In [255]:
wikidates('Q12029491').attributes['claims']['P569'][0]['mainsnak']['datavalue']['value']['time']

'+1979-10-12T00:00:00Z'

In [254]:
wikidates('Q12029491').attributes['claims']['P569'][0]['mainsnak']['datavalue']['value']['time']

'+1979-10-12T00:00:00Z'

### Člověk/těleso

In [22]:
wikidates('Q12029491').attributes['claims']['P31'][0]['mainsnak']['datavalue']['value']['id']

'Q5'

In [23]:
wikidates('Q1266721').attributes['claims']['P31'][0]['mainsnak']['datavalue']['value']['id']

'Q215380'

### Atributy lidí

#### Gender

pohlavní identita subjektu; u lidí použijte muž (Q6581097), žena (Q6581072), transmuž (Q2449503), transžena (Q1052281); u zvířat použijte samec (Q44148), samice (Q43445); u skupin stejného pohlaví použijte „podtřída (čeho)“ (P279)

In [24]:
wikidates('Q12029491').attributes['claims']['P21'][0]['mainsnak']['datavalue']['value']['id']

'Q6581072'

#### Občanství

In [26]:
vsechny_atributy('Q19848','P27')

['Q30']

#### Etnicita

In [27]:
vsechny_atributy('Q19848','P172')

['Q974693', 'Q857880']

#### Náboženství

In [222]:
wikidates('Q19848').attributes['claims']['P140'][0]['mainsnak']['datavalue']['value']['id']

'Q5043'

#### Nástroj

In [29]:
vsechny_atributy('Q19848','P1303')

['Q5994',
 'Q1112687',
 'Q31561',
 'Q78987',
 'Q163829',
 'Q6607',
 'Q128309',
 'Q17172850']

### Atributy kapel

#### Založení

In [30]:
wikidates('Q6774073').attributes['claims']['P571'][0]['mainsnak']['datavalue']['value']['time']

'+1994-00-00T00:00:00Z'

#### Zánik

In [31]:
wikidates('Q1299').attributes['claims']['P576'][-1]['mainsnak']['datavalue']['value']['time']

'+1970-04-10T00:00:00Z'

#### Země původu

In [229]:
vsechny_atributy('Q19848','P19')

['Q6523170']

## A jdem na to!

In [257]:
def stahni_wikidata(slovnik):
    
    slovnik = {k: v for k, v in slovnik.items() if v is not None}
    
    kody = {}
    vysledky = []
    c = 0
    
    def atributy(p):
        atributy = []
        for x in balicek.attributes['claims'][p]:
            atributy.append(x['mainsnak']['datavalue']['value']['id'])
        return(atributy)
    
    def rozkodovani(wikidata_id):
        
        url = f'https://www.wikidata.org/wiki/Special:EntityData/{wikidata_id}.json'
        response = requests.get(url)
        data = response.json()
        try:
            label = data['entities'][wikidata_id]['labels']['cs']['value']
        except:
            label = data['entities'][wikidata_id]['labels']['en']['value']
        kody[wikidata_id] = label
        return label        
    
    def rozkoduj(peckakvecka):
        if isinstance(peckakvecka, list):
            novy_seznam = []
            for x in peckakvecka:
                novy_seznam.append(rozkodovani(x))
            citelne = novy_seznam
        elif isinstance (peckakvecka, str):
            citelne = rozkodovani(peckakvecka)
        else:
            citelne = peckakvecka
        return citelne
        
    for key, value in slovnik.items():
        
        if value != None:
        
            interpret = {}
            interpret['interpret_hlavni'] = key
            interpret['id'] = value
            balicek = wikidates(value)
            
            try:
                interpret['nazev'] = balicek.attributes['claims']['P691'][0]['qualifiers']['P1810'][0]['datavalue']['value']
            except:
                interpret['nazev'] = None
                
            try:
                interpret['zanry'] = rozkoduj(atributy('P136'))
            except:
                interpret['zanry'] = None
            
            try:
                interpret['narozeni'] = balicek.attributes['claims']['P569'][0]['mainsnak']['datavalue']['value']['time']
            except:
                interpret['narozeni'] = None
                
            try:
                interpret['umrti'] = balicek.attributes['claims']['P570'][0]['mainsnak']['datavalue']['value']['time']
            except:
                interpret['umrti'] = None
            
            try:
                interpret['zacatek_pusobeni'] = balicek.attributes['claims']['P2031'][0]['mainsnak']['datavalue']['value']['time']
            except:
                interpret['zacatek_pusobeni'] = None
                
            try:
                interpret['konec_pusobeni'] = balicek.attributes['claims']['P2032'][0]['mainsnak']['datavalue']['value']['time']
            except:
                interpret['konec_pusobeni'] = None
            
            try:
                interpret['spotify_id'] = balicek.attributes['claims']['P1902'][0]['mainsnak']['datavalue']['value']
            except:
                interpret['spotify_id'] = None
                
            try:
                interpret['spotify_odberatele'] = balicek.attributes['claims']['P1902'][0]['qualifiers']['P3744'][0]['datavalue']['value']['amount']
            except:
                interpret['spotify_odberatele'] = None
            
            try:
                interpret['velikost'] = rozkoduj(balicek.attributes['claims']['P31'][0]['mainsnak']['datavalue']['value']['id'])
            except: 
                interpret['velikost'] = None
            
            try:
                interpret['gender'] = rozkoduj(balicek.attributes['claims']['P21'][0]['mainsnak']['datavalue']['value']['id'])
            except:
                interpret['gender'] = None
                
            try:
                interpret['obcanstvi'] = rozkoduj(atributy('P27'))
            except:
                interpret['obcanstvi'] = None
                
            try:
                interpret['etnicita'] = rozkoduj(atributy('P172'))
            except:
                interpret['etnicita'] = None
            
            try:
                interpret['hlas'] = rozkoduj(atributy('P412'))
            except:
                interpret['hlas'] = None
            
            try:
                interpret['nastroje'] = rozkoduj(atributy('P1303'))
            except:
                interpret['nastroje'] = None
            
            try:
                interpret['zalozeni'] = balicek.attributes['claims']['P571'][0]['mainsnak']['datavalue']['value']['time']
            except:
                interpret['zalozeni'] = None
                
            try:
                interpret['zanik'] = balicek.attributes['claims']['P576'][-1]['mainsnak']['datavalue']['value']['time']
            except:
                interpret['zanik'] = None
            
            try:
                interpret['misto_puvodu'] = rozkoduj(atributy('P19')[-1])
            except:                                        
                interpret['misto_puvodu'] = None
                
            try:
                interpret['nabozenstvi'] = rozkoduj(atributy('P140'))
            except:
                interpret['nabozenstvi'] = None
            
            try:
                interpret['orientace'] = rozkoduj(atributy('P91'))
            except:
                interpret['orientace'] = None
            
            c = c + 1
            
            print(f'{c} / {len(slovnik)} / {key}')
            
            vysledky.append(interpret)
            
        else:
            pass
    
    return(vysledky)

In [42]:
import itertools

In [240]:
pokus = { "Ed Sheeran": "Q47447", "Lucie Bílá": "Q468282", "Kabát": "Q938649", "Lady Gaga": "Q19848", "Pharrell Williams": "Q14313", "Michal Hrůza": "Q12036727", "Karel Gott": "Q333187"}

In [ ]:
vsechny_atribu

In [258]:
pokusne_vysledky = stahni_wikidata(pokus)

1 / 7 / Ed Sheeran
2 / 7 / Lucie Bílá
3 / 7 / Kabát
4 / 7 / Lady Gaga
5 / 7 / Pharrell Williams
6 / 7 / Michal Hrůza
7 / 7 / Karel Gott


In [260]:
pokusne_vysledky

[{'interpret_hlavni': 'Ed Sheeran',
  'id': 'Q47447',
  'nazev': None,
  'zanry': ['lidová hudba', 'folk pop', 'pop music', 'folk', 'hip hop', 'rap'],
  'narozeni': '+1991-02-17T00:00:00Z',
  'umrti': None,
  'zacatek_pusobeni': '+2004-00-00T00:00:00Z',
  'konec_pusobeni': None,
  'spotify_id': '6eUKZXaKkcviH0Ku9w2n3V',
  'spotify_odberatele': None,
  'velikost': 'člověk',
  'gender': 'muž',
  'obcanstvi': ['Spojené království'],
  'etnicita': None,
  'hlas': ['tenor'],
  'nastroje': ['kytara', 'hlas', 'live looper'],
  'zalozeni': None,
  'zanik': None,
  'misto_puvodu': 'Halifax',
  'nabozenstvi': None,
  'orientace': None},
 {'interpret_hlavni': 'Lucie Bílá',
  'id': 'Q468282',
  'nazev': None,
  'zanry': ['pop music'],
  'narozeni': '+1966-04-07T00:00:00Z',
  'umrti': None,
  'zacatek_pusobeni': '+1985-00-00T00:00:00Z',
  'konec_pusobeni': None,
  'spotify_id': None,
  'spotify_odberatele': None,
  'velikost': 'člověk',
  'gender': 'žena',
  'obcanstvi': ['Česko'],
  'etnicita': No